In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import yfinance as yf
import datetime

## Import data

In [ ]:
start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2024, 1, 1)
btc_info = yf.Ticker("BTC-USD")

# pass the parameters as the taken dates for start and end
df = btc_info.history(start = start_date, end = end_date)

In [ ]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-01-01 00:00:00+00:00,14112.200195,14112.200195,13154.700195,13657.200195,10291200000,0.0,0.0
2018-01-02 00:00:00+00:00,13625.000000,15444.599609,13163.599609,14982.099609,16846600192,0.0,0.0
2018-01-03 00:00:00+00:00,14978.200195,15572.799805,14844.500000,15201.000000,16871900160,0.0,0.0
2018-01-04 00:00:00+00:00,15270.700195,15739.700195,14522.200195,15599.200195,21783199744,0.0,0.0
2018-01-05 00:00:00+00:00,15477.200195,17705.199219,15202.799805,17429.500000,23840899072,0.0,0.0


In [ ]:
df = df.drop(columns=['Dividends', 'Stock Splits'])

In [ ]:
df.columns = ['open', 'high', 'low', 'close', 'vol']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2191 entries, 2018-01-01 00:00:00+00:00 to 2023-12-31 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    2191 non-null   float64
 1   high    2191 non-null   float64
 2   low     2191 non-null   float64
 3   close   2191 non-null   float64
 4   vol     2191 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 102.7 KB


In [ ]:
# Prepare the volume and price differences, normalize volume
BTC_vol = df["vol"].values
df_diff = df.diff().dropna()
df_diff["vol"] = np.log(1 + BTC_vol[:-1])

In [ ]:
df_aligned = df.loc[df_diff.index]

In [ ]:
# Train data
# Period : From start of 2018 to end of 2022
mask_train = (df_diff.index >= "2018-01-01") & (df_diff.index < "2023-01-01")
df_train = df_diff.loc[mask_train].copy()
train_close = df_aligned.loc[mask_train, "close"].values
df_train["Relative_Close"] = train_close / train_close[0]

In [ ]:
# Test data
# Period : Whole 2023
mask_test = (df_diff.index >= "2023-01-01") & (df_diff.index < "2024-01-01")  # December 2018 for testing
df_test = df_diff.loc[mask_test].copy()
test_close = df_aligned.loc[mask_test, "close"].values
df_test["Relative_Close"] = test_close / train_close[0]


In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
# Generate dataset function
def generate_dataset(df, seq_len):
    X_list, y_list = [], []
    for i in range(len(df) - seq_len):
        X_list.append(df.iloc[i:(i+seq_len), :].values)
        y_list.append(df["close"].iloc[i + seq_len])
    return np.array(X_list), np.array(y_list)

In [ ]:
LAG = 30

In [ ]:
# Prepare training and test datasets
X_train, y_train = generate_dataset(df_train, LAG)
X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [ ]:
# Model architecture
tf.keras.utils.set_random_seed(4002)

model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, validation_split=0.2, verbose=1)

Epoch 1/20
45/45 [==============================] - 3s 19ms/step - loss: 1053101.6250 - val_loss: 1042088.3750
Epoch 2/20
45/45 [==============================] - 1s 13ms/step - loss: 1036544.0000 - val_loss: 1054578.7500
Epoch 3/20
45/45 [==============================] - 1s 22ms/step - loss: 1035534.8125 - val_loss: 1065354.5000
Epoch 4/20
45/45 [==============================] - 1s 20ms/step - loss: 1017603.5000 - val_loss: 1072262.3750
Epoch 5/20
45/45 [==============================] - 1s 20ms/step - loss: 1029645.8125 - val_loss: 1072782.7500
Epoch 6/20
45/45 [==============================] - 1s 14ms/step - loss: 1026412.3750 - val_loss: 1043494.3125
Epoch 7/20
45/45 [==============================] - 1s 14ms/step - loss: 1023574.6250 - val_loss: 1031291.1875
Epoch 8/20
45/45 [==============================] - 1s 14ms/step - loss: 991631.3125 - val_loss: 1056316.5000
Epoch 9/20
45/45 [==============================] - 1s 13ms/step - loss: 992313.6875 - val_loss: 1047305.4375
Epo

In [ ]:
# Predict
predictions = model.predict(X_test)

12/12 [==============================] - 0s 4ms/step


In [ ]:
# Evaluate
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print(f'Test MSE: {mse}')

Test MSE: 439152.70355785725
